In [109]:
import pandas as pd
import numpy as np
import random
import copy


from sklearn.model_selection import KFold 
from sklearn.model_selection import train_test_split

### Data Preparation

In [6]:
movies_df = pd.read_csv('movies.dat', header = None, sep='::', names =['MovieID', 'Title', 'Genres'],
                    engine='python')
users_df = pd.read_csv('users.dat',  header = None, sep='::', 
                    engine='python')
ratings_df = pd.read_csv('ratings.dat',  header = None, sep='::', names = ['UserID', 'MovieID', 'Rating', 'Timestamp'],
                    engine='python')

In [111]:
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R = R_df.as_matrix()
R_idx = np.argwhere(R!=0)
R_train_idx, R_test_idx = train_test_split(R_idx, test_size = 0.2)

In [113]:
R_train = R.copy()
R_test = R.copy()
R_train[R_test_idx[:, 0], R_test_idx[:, 1]] = 0
R_test[R_train_idx[:, 0], R_train_idx[:, 1]] = 0

In [116]:
R_train

array([[ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 3.,  0.,  0., ...,  0.,  0.,  0.]])

In [91]:
# train_test_split = 0.9
# U, I  = np.shape(R)
# R_train = copy.deepcopy(R)
# R_train[:,int(I*train_test_split):] = 0
# R_train

array([[ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 3.,  0.,  0., ...,  0.,  0.,  0.]])

### Model Class

In [101]:
def r_hat (mu, P, Q, bu, bi):
    temp = np.dot(P, Q.T) + mu
    temp_u = np.array([(x + bu.T).ravel() for x in temp.T]).T
    return np.array([(x + bi.T).ravel() for x in temp_u])

def loss_function(R, P, Q, F, bu, bi, beta):
    e = 0
    N = np.count_nonzero(R)
    mu = np.sum(np.sum(R, axis = 1)) / N
    test_idx = R == 0
    R_hat = r_hat(mu, P, Q, bu, bi)
    R_hat[test_idx] = 0
    # loss function error sum( (y-y_hat)^2 )
    error = (R - R_hat)**2
    e = sum(np.sum(error, axis = 1))
    if F > 0:
        e += beta/2.0 *(sum([sum(n * p) for n, p in zip(np.count_nonzero(error, axis = 1), P)]))
        e += beta/2.0 *(sum([sum(n * q) for n, q in zip(np.count_nonzero(error, axis = 0), Q)]))
#     regularization = np.dot (P**2, (Q**2).T)
#     regularization[test_idx] = 0
#     e += beta/2.0*(sum(np.sum(regularization, axis = 1)))
    return e/N

def loss_gradient(R, P, Q, F, bu, bi, learning_rate=0.0001, penalty = 0.02):
    N = np.count_nonzero(R)
    mu = np.sum(np.sum(R, axis = 1)) / N
    test_idx = R == 0
    R_hat = r_hat(mu, P, Q, bu, bi)
    R_hat[test_idx] = 0
    e = (R - R_hat)
    
    grad_bu = - np.sum(e,axis = 1).reshape(-1,1)
    grad_bi = - np.sum(e, axis = 0).reshape(-1,1)
    
    grad_P = np.zeros((len(P), F))
    grad_Q = np.zeros((len(Q), F))
    if F > 0:
#         grad_P = np.array([np.array([-np.dot(r, q) for q in Q.T]) + penalty * p for r, p in zip(e, P)])
#         grad_Q = np.array([np.array([-np.dot(r, p) for p in P.T]) + penalty * q for r, q in zip(e.T, Q)])
        grad_P = - np.dot(e, Q) + penalty * P
        grad_Q = - np.dot(e.T, P) + penalty * Q
    
#     if F > 0:
#         train_idx = np.argwhere(e!=0)
#         for idx in train_idx:
#             u, i = idx
#             P[u] += learning_rate*(e[u, i]*Q[i] - penalty*P[u])
#             Q[i] += learning_rate*(e[u, i]*P[u] - penalty*Q[i])
    return grad_P, grad_Q, grad_bu, grad_bi
                
                

In [93]:
def report_function(e, R, P, Q, F, bu, bi, beta):
    U = R.shape[0]
    perm = np.random.choice(R.shape[0],U)
    loss = loss_function(R[perm], P[perm], Q, F, bu[perm], bi, beta)
    N = np.count_nonzero(R[perm])
    mu = np.sum(np.sum(R[perm], axis = 1)) / N
    idx = R[perm] == 0
    l = r_hat(mu, P[perm], Q, bu[perm], bi)
    l[idx] = 0
    err=(l-R[perm])**2
    l[idx] = 1
    err/=l
    train_dispersion=np.mean(np.mean(err, axis = 1))
    print("\t",e,"Loss =",loss,"Train_Dispersion",train_dispersion,end="\t")
    print()


def optimization_SGD(R, P, Q, F, bu, bi, 
                       learning_rate=0.0001,
                       penalty = 0.02,
                       tol=1e-8, 
                       max_iter=1000, 
                       batch_size=100,
                       verbose = True):
    N = R.shape[0]
    l0 = loss_function(R, P, Q, F, bu, bi, penalty)
    
    for e in range(max_iter):
        if (e%(max_iter//10)==0 and verbose): ##train a small batches of data
            report_function(e, R, P, Q, F, bu, bi, penalty)
        perm = np.random.permutation(N)
        for i in range(0,N,batch_size):
            Rb = R[perm[i:i+batch_size]]
            Pb = P[perm[i:i+batch_size]]
            bub = bu[perm[i:i+batch_size]]
#             print (np.shape(P[perm[i:i+batch_size]]), np.shape(Q),np.shape(bub),np.shape(bi))
            grad_P, grad_Q, grad_bu, grad_bi = loss_gradient(Rb, Pb, Q, F, bub, bi, learning_rate, penalty)
            
            #update P, Q, bu, bi
            P[perm[i:i+batch_size]] -= learning_rate*grad_P
            bu[perm[i:i+batch_size]] -= learning_rate*grad_bu
            
            Q -= learning_rate*grad_Q
            bi -= learning_rate*grad_bi
            
        l = loss_function(R, P, Q, F, bu, bi, penalty)
       
        d=np.abs(l-l0)
        if d<tol*l0:
            break
        l0=l  
    if verbose:
        report_function(e, R, P, Q, F, bu, bi, penalty)
        
    return P, Q, bu, bi
    
    

In [94]:
class RecommenderModel(object):
    def __init__(self, 
                 F = 0,
                 learning_rate=0.001,
                 penalty = 0.02,
                 tol=1e-8, 
                 max_iter=1000, 
                 batch_size=100,
                 verbose = True):
        self.mu = None
        self.Q = None
        self.P = None
        self.bu = None
        self.bi = None
        
        ## initialize hyper parameters
        self.F = F
        self.learning_rate = learning_rate
        self.penalty = penalty
        self.tol = tol
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.verbose = verbose
        
    
    def fit_matrix_factorization_model(self, R):
        """
        Return the fitted model
        @param: matrix factor
        """
        U = len(R)
        I = len(R[0])   
        F = self.F
        
        ## Generate user bias bu U coefficient
        ## Generte item bias bi I coefficient
        bu = np.random.normal(0, 1e-4, (U, 1))
        bi = np.random.normal(0, 1e-4, (I, 1))
        
        # Generate P - U x F
        # Use random values to start
        P = np.random.normal(0, 1/max(1, F**0.5), (U, F))

        # Generate Q - I x F
        # Use random values to start
        Q = np.random.normal(0, 1/max(1, F**0.5), (I, F))
        
        # Calculate mu 
        N = np.count_nonzero(R)
        self.mu = np.sum(np.sum(R, axis = 1)) / N

        P, Q, bu, bi = optimization_SGD(R, P, Q, F, bu, bi, 
                                        self.learning_rate, 
                                        self.penalty,
                                        self.tol,
                                        self.max_iter,
                                        self.batch_size,
                                        self.verbose)

        self.Q = Q
        self.P = P
        self.bu = bu
        self.bi = bi
        return self
        
    def predict_instance(self, p_index):
        """
        Returns all predictions for a given row
        @param p_index:
        @return:
        """
        return self.mu + self.bu[p_index] + self.bi.T + np.dot(self.P[p_index, :], self.Q.T)

    def predict_all(self):
        """
        @Returns the full prediction matrix
        @param:
        @return:
        """
        temp = np.dot(self.P, self.Q.T) + self.mu
        temp_u = np.array([(x + self.bu.T).ravel() for x in temp.T]).T
        return np.array([(x + self.bi.T).ravel() for x in temp_u])   

In [126]:
learning_rates = np.arange(0.0001, 0.001, 0.0004)
F = 0

In [128]:
train_error = []
test_error = []
for learning_rate in learning_rates:
    model = RecommenderModel(learning_rate = learning_rate, tol=1e-5, max_iter = 500, F = F, batch_size = 100, penalty = 0.02)
    model.fit_matrix_factorization_model(R_train)
    test_idx = R == 0
    R_hat = model.predict_all()
    R_hat[test_idx] = 0
    all_error = (R - R_hat)**2
    train_error += [np.sum(all_error[R_train_idx[:, 0], R_train_idx[:, 1]])/ len(R_train_idx)]         
    test_error += [np.sum(all_error[R_test_idx[:, 0], R_test_idx[:, 1]])/ len(R_test_idx)]   
    print ("train error (learning_rate = {0}, F = {1}): ".format(learning_rate, F), train_error[-1])
    print ("test error (learning_rate = {0}, F = {1}): ".format(learning_rate, F), test_error[-1])

	 0 Loss = 1.24437397182 Train_Dispersion 0.012478633477	
	 50 Loss = 0.880062970547 Train_Dispersion 0.00913554662111	
	 100 Loss = 0.833213426462 Train_Dispersion 0.00861943946355	
	 150 Loss = 0.831187109973 Train_Dispersion 0.00868988075443	
	 200 Loss = 0.816706659024 Train_Dispersion 0.00865093217561	
	 250 Loss = 0.815886150747 Train_Dispersion 0.00856140611781	
	 300 Loss = 0.80889776851 Train_Dispersion 0.00858964528319	
	 350 Loss = 0.814526861521 Train_Dispersion 0.00857552641327	
	 371 Loss = 0.818722688218 Train_Dispersion 0.00854060188511	
train error (learning_rate = 0.0001, F = 0):  0.809778006563
test error (learning_rate = 0.0001, F = 0):  0.823978880617
	 0 Loss = 1.2352371647 Train_Dispersion 0.0125277283037	
	 50 Loss = 0.82118009325 Train_Dispersion 0.00870655718559	
	 72 Loss = 0.823241082419 Train_Dispersion 0.00885311381979	
train error (learning_rate = 0.0005, F = 0):  0.810287865905
test error (learning_rate = 0.0005, F = 0):  0.824479256409
	 0 Loss = 1.2606

In [129]:
best_idx = np.argmin(test_error)
best_learning_rate = learning_rates[best_idx]
print ("best train error (penalty = {0}, F = {1}): ".format(best_learning_rate, F), train_error[best_idx])
print ("best test error (penalty = {0}, F = {1}): ".format(best_learning_rate, F), test_error[best_idx])

best train error (penalty = 0.0001, F = 0):  0.809778006563
best test error (penalty = 0.0001, F = 0):  0.823978880617


### Model selection

In [130]:
kf=KFold(5,shuffle=True)
folds=list(kf.split(R_train_idx))

In [138]:
def model_cross_validation(model, R, folds):
    kfolds=len(folds)
    train_performance=np.empty(kfolds)
    validation_performance=np.empty(kfolds)
    for idx in range(kfolds):
        train_idx,validation_idx=folds[idx]
        R_train_cv = R_train.copy()
        R_val_cv = R_train.copy()
        R_train_cv[R_train_idx[validation_idx][:,0], R_train_idx[validation_idx][:,1]] = 0
        R_val_cv[R_train_idx[train_idx][:,0], R_train_idx[train_idx][:,1]] = 0
        N_train = len(train_idx)
        N_val = len(validation_idx)
        ##fit the model
        model.fit_matrix_factorization_model(R_train_cv)
        test_idx = R_train == 0
        R_hat_cv = model.predict_all()  
        R_hat_cv[test_idx] = 0
        all_error = (R_train - R_hat_cv)**2    
        train_error_curr = np.sum(all_error[R_train_idx[train_idx][:,0], R_train_idx[train_idx][:,1]])/N_train       
        validation_error_curr = np.sum(all_error[R_train_idx[validation_idx][:,0], R_train_idx[validation_idx][:,1]])/N_val
        train_performance[idx] = train_error_curr
        validation_performance[idx] = validation_error_curr
    return np.array(train_performance),np.array(validation_performance)

In [139]:
train=[]
train_std=[]
validation=[]
validation_std=[]
models_params = [[2, best_learning_rate, 0.02, 1e-5, 500, 100],[4, best_learning_rate, 0.04, 1e-5, 500, 500],[6, best_learning_rate, 0.1, 1e-5, 1000, 1000]]
models = []
for i in range(len(models_params)):
    F,learning_rate, penalty, tol, max_iter, batch_size=models_params[i]
    models += [RecommenderModel(F,learning_rate, penalty, tol, max_iter, batch_size)]
    t,v = model_cross_validation(models[i], R_train, folds)
    train.append(t.mean())
    train_std.append(t.std())
    validation.append(v.mean())
    validation_std.append(v.std())
    print ("----------------------------------------------------------------------------------")
    print("F={0}, learning_rate = {1}, penalty={2}, max_iter = {3}, batch_size = {4} :".format(F, learning_rate, penalty, max_iter, batch_size))
    print(t.mean(),v.mean(),t.std(),v.std())
    print ("==================================================================================")
validation=np.array(validation)
train=np.array(train)

	 0 Loss = 1.74107824236 Train_Dispersion 0.0159397890023	
	 50 Loss = 0.96040523319 Train_Dispersion 0.00825362681046	
	 100 Loss = 0.876979948411 Train_Dispersion 0.00748258249428	
	 150 Loss = 0.844405399665 Train_Dispersion 0.00712810119156	
	 200 Loss = 0.823292802073 Train_Dispersion 0.0069247151007	
	 250 Loss = 0.82236151957 Train_Dispersion 0.00688299874038	
	 300 Loss = 0.818463163773 Train_Dispersion 0.00694437306877	
	 350 Loss = 0.813491117228 Train_Dispersion 0.00685812871573	
	 400 Loss = 0.811205344055 Train_Dispersion 0.00683173480101	
	 450 Loss = 0.798905658648 Train_Dispersion 0.00659912677241	
	 499 Loss = 0.791467321336 Train_Dispersion 0.00688537541793	
	 0 Loss = 1.75394272854 Train_Dispersion 0.0175465414227	
	 50 Loss = 0.953263477503 Train_Dispersion 0.00820843110221	
	 100 Loss = 0.867261448125 Train_Dispersion 0.0073829723417	
	 150 Loss = 0.844989728499 Train_Dispersion 0.00725442351552	
	 200 Loss = 0.833938044061 Train_Dispersion 0.00699173716763	
	 250 

	 100 Loss = 0.868328006106 Train_Dispersion 0.00723880082685	
	 200 Loss = 0.82273295042 Train_Dispersion 0.00687175987892	
	 300 Loss = 0.806342913475 Train_Dispersion 0.00681930900505	
	 400 Loss = 0.805986459603 Train_Dispersion 0.00668889998375	
	 500 Loss = 0.781538126983 Train_Dispersion 0.00676281157289	
	 600 Loss = 0.773911477593 Train_Dispersion 0.00651475191818	
	 700 Loss = 0.771675596839 Train_Dispersion 0.0066571433895	
	 800 Loss = 0.752022869387 Train_Dispersion 0.00655782192134	
	 900 Loss = 0.750370516525 Train_Dispersion 0.00634626112218	
	 999 Loss = 0.7353540898 Train_Dispersion 0.0063389662454	
	 0 Loss = 1.42459499338 Train_Dispersion 0.0114108192863	
	 100 Loss = 0.878947398515 Train_Dispersion 0.00745350716547	
	 200 Loss = 0.824187953877 Train_Dispersion 0.00708111074123	
	 300 Loss = 0.811690815986 Train_Dispersion 0.00708354051436	
	 400 Loss = 0.788077128965 Train_Dispersion 0.00662902875988	
	 500 Loss = 0.782568029665 Train_Dispersion 0.00661822507397	
	

In [142]:
best_model_idx = np.argmin(validation)
best_model = models[best_model_idx]
best_model.fit_matrix_factorization_model(R_train)
best_R_hat = best_model.predict_all()
best_R_hat[test_idx] = 0
best_all_error = (R - best_R_hat)**2
best_train_error = np.sum(best_all_error[R_train_idx[:, 0], R_train_idx[:,1]])/ len(R_train_idx)
best_test_error = np.sum(best_all_error[R_test_idx[:, 0], R_test_idx[:,1]])/ len(R_test_idx)
best_F,best_learning_rate, best_penalty, best_tol, best_max_iter, best_batch_size=models_params[best_model_idx]
print("best model hyperparamters: ")
print("F={0}, learning_rate = {1}, penalty={2}, max_iter = {3}, batch_size = {4} :".format(best_F, best_learning_rate, best_penalty, best_max_iter, best_batch_size))
print()
print ("train error (learning_rate = {0}, F = {1}): ".format(learning_rate, F), train_error[-1])
print ("test error (learning_rate = {0}, F = {1}): ".format(learning_rate, F), test_error[-1])

	 0 Loss = 1.40229490187 Train_Dispersion 0.0141176405261	
	 100 Loss = 0.857359090466 Train_Dispersion 0.00898123379792	
	 200 Loss = 0.808959173562 Train_Dispersion 0.00876305658539	
	 300 Loss = 0.800951060107 Train_Dispersion 0.00815417343371	
	 400 Loss = 0.777728966391 Train_Dispersion 0.00832153337415	
	 500 Loss = 0.761345788573 Train_Dispersion 0.00829499159712	
	 600 Loss = 0.755177612104 Train_Dispersion 0.00830584901377	
	 700 Loss = 0.735881066941 Train_Dispersion 0.00793664494536	
	 800 Loss = 0.727155364678 Train_Dispersion 0.00768154584249	
	 900 Loss = 0.715223609737 Train_Dispersion 0.00790154388885	
	 999 Loss = 0.710000580562 Train_Dispersion 0.00774103340954	
best model hyperparamters: 
F=6, learning_rate = 0.0001, penalty=0.1, max_iter = 1000, batch_size = 1000 :

train error (learning_rate = 0.0001, F = 6):  0.810868286915
test error (learning_rate = 0.0001, F = 6):  0.825195540061
